In [2]:
from src.data.data_loader import load_data

# Specify the columns to keep
keep_columns = [
    "id",
    "p_num",
    "time",
    "bg-0:00",
    "insulin-0:00",
    "carbs-0:00",
    "hr-0:00",
    "steps-0:00",
    "cals-0:00",
    "activity-0:00",
]



# Load the data with specified columns
df = load_data(
    data_source_name="kaggle_brisT1D", dataset_type="train", keep_columns=keep_columns
)

# Print the first 5 rows of the data
print(df.head())

      id p_num      time  bg-0:00  insulin-0:00  carbs-0:00  hr-0:00  \
0  p01_0   p01  06:10:00     15.1        0.0417         NaN      NaN   
1  p01_1   p01  06:25:00     14.4        0.0417         NaN      NaN   
2  p01_2   p01  06:40:00     13.9        0.0417         NaN      NaN   
3  p01_3   p01  06:55:00     13.8        0.0417         NaN      NaN   
4  p01_4   p01  07:10:00     13.4        0.0417         NaN      NaN   

   steps-0:00  cals-0:00 activity-0:00  
0         NaN        NaN           NaN  
1         NaN        NaN           NaN  
2         NaN        NaN           NaN  
3         NaN        NaN           NaN  
4         NaN        NaN           NaN  


/home/cjrisi/Projects/diabetes/nocturnal-hypo-gly-prob-forecast/src/data/data_loader.py:31: DtypeWarning: Columns (506) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path, usecols=keep_columns)



# Probabilistic Forecasting with `sktime`: Distribution Forecast

## Notebook Goal: 
* Provide an example workflow of a distribution forecast with BGL data using sktime.
* Demonstrate how to use Optuna for a single forecasting type.

In [3]:
%load_ext autoreload
%autoreload 2

## Data Load

In [24]:
from src.data.data_loader import load_data

# Specify the columns to keep
keep_columns = [
    "id",
    "p_num",
    "time",
    "bg-0:00",
    "insulin-0:00",
    "carbs-0:00",
    "hr-0:00",
    "steps-0:00",
    "cals-0:00",
    "activity-0:00",
]

# Load the data with specified columns
df = load_data(
    data_source_name="kaggle_brisT1D", dataset_type="train"
)

# Print the first 5 rows of the data
print(df.head())

/home/cjrisi/Projects/diabetes/nocturnal-hypo-gly-prob-forecast/src/data/data_loader.py:31: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path, usecols=keep_columns)


      id p_num      time  bg-5:55  bg-5:50  bg-5:45  bg-5:40  bg-5:35  \
0  p01_0   p01  06:10:00      NaN      NaN      9.6      NaN      NaN   
1  p01_1   p01  06:25:00      NaN      NaN      9.7      NaN      NaN   
2  p01_2   p01  06:40:00      NaN      NaN      9.2      NaN      NaN   
3  p01_3   p01  06:55:00      NaN      NaN      8.7      NaN      NaN   
4  p01_4   p01  07:10:00      NaN      NaN      8.4      NaN      NaN   

   bg-5:30  bg-5:25  ...  activity-0:40  activity-0:35  activity-0:30  \
0      9.7      NaN  ...            NaN            NaN            NaN   
1      9.2      NaN  ...            NaN            NaN            NaN   
2      8.7      NaN  ...            NaN            NaN            NaN   
3      8.4      NaN  ...            NaN            NaN            NaN   
4      8.1      NaN  ...            NaN            NaN            NaN   

   activity-0:25  activity-0:20  activity-0:15  activity-0:10  activity-0:05  \
0            NaN            NaN           

In [25]:
from sktime.forecasting.arima import ARIMA

# step 1: data specification
y = df["bg-0:00"]
# step 2: specifying forecasting horizon
fh = [1, 2, 3]
# step 3: specifying the forecasting algorithm
forecaster = ARIMA()
# step 4: fitting the forecaster
forecaster.fit(y, fh=[1, 2, 3])
# step 5: querying predictions
y_pred = forecaster.predict()

# for probabilistic forecasting:
#   call a probabilistic forecasting method after or instead of step 5
y_pred_int = forecaster.predict_interval(coverage=0.9)
y_pred_int

bg-0:00           
              0.9           
            lower      upper
177024  10.782319  12.334859
177025  10.426719  12.608672
177026  10.149430  12.805196

In [26]:
df_row = df[df["id"] == 'p01_0']

activity_columns = [col for col in df.columns if 'activity' in col]
bg_columns = [col for col in df.columns if 'bg-' in col]
insulin_columns = [col for col in df.columns if 'insulin' in col]
carbs_columns = [col for col in df.columns if 'carbs' in col]
steps_columns = [col for col in df.columns if 'steps' in col]
cals_columns = [col for col in df.columns if 'cals' in col]
hr_columns = [col for col in df.columns if 'hr' in col]

In [16]:
bg_columns

['bg-5:55',
 'bg-5:50',
 'bg-5:45',
 'bg-5:40',
 'bg-5:35',
 'bg-5:30',
 'bg-5:25',
 'bg-5:20',
 'bg-5:15',
 'bg-5:10',
 'bg-5:05',
 'bg-5:00',
 'bg-4:55',
 'bg-4:50',
 'bg-4:45',
 'bg-4:40',
 'bg-4:35',
 'bg-4:30',
 'bg-4:25',
 'bg-4:20',
 'bg-4:15',
 'bg-4:10',
 'bg-4:05',
 'bg-4:00',
 'bg-3:55',
 'bg-3:50',
 'bg-3:45',
 'bg-3:40',
 'bg-3:35',
 'bg-3:30',
 'bg-3:25',
 'bg-3:20',
 'bg-3:15',
 'bg-3:10',
 'bg-3:05',
 'bg-3:00',
 'bg-2:55',
 'bg-2:50',
 'bg-2:45',
 'bg-2:40',
 'bg-2:35',
 'bg-2:30',
 'bg-2:25',
 'bg-2:20',
 'bg-2:15',
 'bg-2:10',
 'bg-2:05',
 'bg-2:00',
 'bg-1:55',
 'bg-1:50',
 'bg-1:45',
 'bg-1:40',
 'bg-1:35',
 'bg-1:30',
 'bg-1:25',
 'bg-1:20',
 'bg-1:15',
 'bg-1:10',
 'bg-1:05',
 'bg-1:00',
 'bg-0:55',
 'bg-0:50',
 'bg-0:45',
 'bg-0:40',
 'bg-0:35',
 'bg-0:30',
 'bg-0:25',
 'bg-0:20',
 'bg-0:15',
 'bg-0:10',
 'bg-0:05',
 'bg-0:00']

In [33]:
keep_columns = ['id', 'p_num','time']
keep_columns.extend(bg_columns)

df_bg = df_row[keep_columns]


In [45]:
df_bg.melt(id_vars=['id', 'p_num', 'time'], var_name='bg_time', value_name='bg_value')

df_insulin = df_row[keep_columns]
df_insulin.melt(id_vars=['id', 'p_num', 'time'], var_name='insulin_time', value_name='insulin_value')

df_carbs = df_row[keep_columns]
df_carbs.melt(id_vars=['id', 'p_num', 'time'], var_name='carbs_time', value_name='carbs_value')

df_steps = df_row[keep_columns]
df_steps.melt(id_vars=['id', 'p_num', 'time'], var_name='steps_time', value_name='steps_value')

df_cals = df_row[keep_columns]
df_cals.melt(id_vars=['id', 'p_num', 'time'], var_name='cals_time', value_name='cals_value')

df_hr = df_row[keep_columns]
df_hr.melt(id_vars=['id', 'p_num', 'time'], var_name='hr_time', value_name='hr_value')


,id,p_num,time,bg_time,bg_value
0,p01_0,p01,06:10:00,bg-5:55,NaN
1,p01_0,p01,06:10:00,bg-5:50,NaN
2,p01_0,p01,06:10:00,bg-5:45,9.6
3,p01_0,p01,06:10:00,bg-5:40,NaN
4,p01_0,p01,06:10:00,bg-5:35,NaN
...,...,...,...,...,...
67,p01_0,p01,06:10:00,bg-0:20,NaN
68,p01_0,p01,06:10:00,bg-0:15,16.2
69,p01_0,p01,06:10:00,bg-0:10,NaN
70,p01_0,p01,06:10:00,bg-0:05,NaN


In [ ]:
>>> df.melt(id_vars=['A'], value_vars=['B'],
...         var_name='myVarname', value_name='myValname')
   A myVarname  myValname
0  a         B          1
1  b         B          3
2  c         B          5

In [ ]:
5:50 | 5:45 | 5:40

bg: 1 | 2 | 3
insulin: 4 | 5 | 6


SyntaxError: illegal target for annotation (3543345743.py, line 1)

In [7]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

In [8]:
import pandas as pd

df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},

                   'B': {0: 1, 1: 3, 2: 5},

                   'C': {0: 2, 1: 4, 2: 6}})

df


,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6


In [32]:
# df.melt(id_vars=['A'], value_vars=['valA-5:55', 'valA-0:00'])

bg_col = df.melt(id_vars=['id'], value_vars=bg_columns)
bg_col

,id,variable,value
0,p01_0,bg-5:55,NaN
1,p01_1,bg-5:55,NaN
2,p01_2,bg-5:55,NaN
3,p01_3,bg-5:55,NaN
4,p01_4,bg-5:55,NaN
...,...,...,...
12745723,p12_25294,bg-0:00,9.8
12745724,p12_25295,bg-0:00,10.3
12745725,p12_25296,bg-0:00,10.7
12745726,p12_25297,bg-0:00,11.0
